In [1]:
import hopsworks
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import datetime
import os

In [ ]:
# Uncomment this cell and fill in details if you are running your own Hopsworks cluster
# key=""
# with open("api-key.txt", "r") as f:
#     key = f.read().rstrip()
# os.environ['HOPSWORKS_PROJECT']="cjsurf"
# os.environ['HOPSWORKS_HOST']="35.187.178.84"
# os.environ['HOPSWORKS_API_KEY']=key

In [2]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/135
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
mr = project.get_model_registry()
model = mr.get_model("cjsurf_model", version=1)
model_dir = model.download()

Connected. Call `.close()` to terminate connection gracefully.

In [4]:
import joblib
model = joblib.load(model_dir + "/knn_model.pkl")

In [5]:
fv = fs.get_feature_view("lahinch_surf", version=1)
fv.init_serving(1)

In [ ]:
key = {"beach_id" : 1}
vector = fv.get_feature_vector(key)
print(vector)

In [ ]:
feature_vectors = []
# Remove the 'pred_dtime' column
feature_vectors.append(vector[0:3])
for i in range(4,len(vector),4):
    feature_vectors.append(vector[i:i+3])
# 'pred_dtime'
dt = vector[3]
feature_vectors

In [ ]:
row_list=[]
columns=["beach_id","hits_at", "wave_height"]
for i in range(0,len(feature_vectors)):
    arr=[]
    arr.append(feature_vectors[i])
    res = model.predict(arr)
    row = []
    row.append(1) #beach_id
    hour = datetime.timedelta(0, (1*3600*i))
    ts = dt + hour
    dt_str = ts.strftime("%Y-%m-%d %H:%M") #:%S
    row.append(dt_str)
    row.append(res[0])
    row_list.append(row)

row_list

In [ ]:
import pandas as pd

df = pd.DataFrame(row_list, columns=columns)

df.info()

In [ ]:
fs = project.get_feature_store()

preds_fg = fs.get_or_create_feature_group(name="wave_predictions",
                version=1,
                primary_key=["beach_id", "hits_at"],
                description="Lahinch surf height predictions",
                online_enabled=True
                )
preds_fg.insert(df)   

In [ ]:
import plotly.express as px
fig = px.line(df, x = "hits_at", y = "wave_height", markers=True, 
              title = "Wave Heights at Lahinch")
fig.update_layout(
#    showlegend=False,
    plot_bgcolor="white",
    margin=dict(t=50,l=10,b=10,r=10)
)

# fig.update_layout(
#     xaxis_tickformat = '%y/%m/%y<br>%Y'
# )

#fig.update_xaxes(minor=dict(ticklen=6, tickcolor="black", showgrid=True))
#fig.update_yaxes(showgrid=False)
#fig.update_xaxes(showgrid=False)
fig.update_xaxes(dtick=12)
fig.show()
#fig.show(config=dict(displayModeBar=False))

In [ ]:
filename="latest_lahinch.png"
fig.write_image(file=filename, format="png", width=1920, height=1280)
dataset_api = project.get_dataset_api()
uploaded_file_path = dataset_api.upload(filename, "Resources", overwrite=True)

# update images with predictions of previous week and outcomes